In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [53]:
ol = pd.read_csv('olympics.csv', index_col = 0, skiprows=1)
pobl = pd.read_csv('poblacion.csv', index_col = 0, skiprows=4) # deben empezar en la fila 5 
PIB = pd.read_csv('PIB.csv', index_col = 0, skiprows=4)


## Dataframe olympics (ol) 

In [54]:
ol.head(n=3)

,№ Summer,01 !,02 !,03 !,Total,№ Winter,01 !.1,02 !.1,03 !.1,Total.1,№ Games,01 !.2,02 !.2,03 !.2,Combined total
Afghanistan (AFG),13,0,0,2,2,0,0,0,0,0,13,0,0,2,2
Algeria (ALG),12,5,2,8,15,3,0,0,0,0,15,5,2,8,15
Argentina (ARG),23,18,24,28,70,18,0,0,0,0,41,18,24,28,70


## Dataframe población (pobl)

In [55]:
pobl.head(n=2)


,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,Unnamed: 62
Country Name,,,,,,,,,,,,,,,,,,,,,
Aruba,ABW,"Population, total",SP.POP.TOTL,54211.0,55438.0,56225.0,56695.0,57032.0,57360.0,57715.0,...,101453.0,101669.0,102053.0,102577.0,103187.0,103795.0,104341.0,104822.0,NaN,NaN
Afghanistan,AFG,"Population, total",SP.POP.TOTL,8996351.0,9166764.0,9345868.0,9533954.0,9731361.0,9938414.0,10152331.0,...,28004331.0,28803167.0,29708599.0,30696958.0,31731688.0,32758020.0,33736494.0,34656032.0,NaN,NaN


## Dataframe PIB 

In [81]:
PIB.head(n=2)


,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
Country Name,,,,,,,,,,,,,,,,,,,,,
Aruba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.791961e+09,2.498933e+09,2.467704e+09,2.584464e+09,NaN,NaN,NaN,NaN,NaN,NaN
Afghanistan,5.377778e+08,5.488889e+08,5.466667e+08,7.511112e+08,8.000000e+08,1.006667e+09,1.400000e+09,1.673333e+09,1.373333e+09,1.408889e+09,...,1.019053e+10,1.248694e+10,1.593680e+10,1.793024e+10,2.053654e+10,2.026425e+10,2.061610e+10,1.921556e+10,1.946902e+10,NaN


### Columnas más legibles dataframe olympics

In [57]:
for col in ol.columns:
    if col[:2]=='01':
        ol.rename(columns={col:'Gold' + col[4:]}, inplace=True)
    if col[:2]=='02':
        ol.rename(columns={col:'Silver' + col[4:]}, inplace=True)
    if col[:2]=='03':
        ol.rename(columns={col:'Bronze' + col[4:]}, inplace=True)

In [58]:
ol.head(n=2)

,№ Summer,Gold,Silver,Bronze,Total,№ Winter,Gold.1,Silver.1,Bronze.1,Total.1,№ Games,Gold.2,Silver.2,Bronze.2,Combined total
Afghanistan (AFG),13,0,0,2,2,0,0,0,0,0,13,0,0,2,2
Algeria (ALG),12,5,2,8,15,3,0,0,0,0,15,5,2,8,15


### Eliminado el country code de los indices del dataset olympics para poder mas adelante hacer un merge con los dataframe PIB y poblacion 

In [59]:
x=0            # necesario para poder hacer un merge que los indices del dataframe sean identicos
for fil in ol.index:
    if '(' in fil:
        f=ol.index[x]
        ol.rename(index={fil: f[:f.find("(")-1]}, inplace=True)
    x+=1      

In [60]:
ol.head(n=2)

,№ Summer,Gold,Silver,Bronze,Total,№ Winter,Gold.1,Silver.1,Bronze.1,Total.1,№ Games,Gold.2,Silver.2,Bronze.2,Combined total
Afghanistan,13,0,0,2,2,0,0,0,0,0,13,0,0,2,2
Algeria,12,5,2,8,15,3,0,0,0,0,15,5,2,8,15


### quitar columnas del dataframe PIB y poblacion que no son necesarios

In [61]:
PIB=PIB.drop('Country Code',axis=1)
PIB=PIB.drop('Indicator Name',axis=1)
PIB=PIB.drop('Indicator Code',axis=1)
PIB=PIB.drop('Unnamed: 62',axis=1)
pobl=pobl.drop('Country Code',axis=1)
pobl=pobl.drop('Indicator Name',axis=1)
pobl=pobl.drop('Indicator Code',axis=1)
pobl=pobl.drop('Unnamed: 62',axis=1)

### Haciendo la media del PIB para cada pais 

In [62]:
s_PIB=PIB.loc[PIB.index].mean(axis=1)    # axis=1 necesario para que sume horizontalmente
s_pobl=pobl.loc[pobl.index].mean(axis=1)
s_pobl=pd.DataFrame(s_pobl)              # volviendo dataframe porque por defecto retorna una Series
s_PIB=pd.DataFrame(s_PIB)

In [63]:
s_pobl.rename(columns={0: "Poblacion media"}, inplace=True)  # renombrando las columnas
s_PIB.rename(columns={0: "promedio PIB"}, inplace=True)
s_pobl.head(n=2)

,Poblacion media
Country Name,
Aruba,7.471237e+04
Afghanistan,1.704045e+07


## Utilizando uno de los sistemas de clasificación; Fibonacci weighted point system (3:2:1)  

gold 3 points, silver 2 points, and bronze 1 point. Se asignan pesos a las medallas para luego sumar y dividir sobre el total de medallas

In [64]:
summer_med=(ol['Gold']*3+ol['Silver']*2+ol['Bronze'])/ol['Total']           # Juegos de verano
winter_med=(ol['Gold.1']*3+ol['Silver.1']*2+ol['Bronze.1'])/ol['Total.1']   # Juegos de invierno
total_med=(ol['Gold.2']*3+ol['Silver.2']*2+ol['Bronze.2'])/ol['Combined total']  # Total de invierno + verano
summer_med=pd.DataFrame(summer_med)    # volviendo dataframe
winter_med=pd.DataFrame(winter_med)
total_med=pd.DataFrame(total_med)

## Haciendo un merge entre los dataframe de juegos de summer, winter y total con los pesos ya asignados previamente

In [65]:
ol_general=pd.merge(summer_med,winter_med,how='outer',left_index=True,right_index=True)
ol_general=pd.merge(ol_general,total_med,how='outer',left_index=True,right_index=True)
ol_general.rename(columns={'0_x': "mean Summer"}, inplace=True)
ol_general.rename(columns={'0_y': "mean Winter"}, inplace=True)
ol_general.rename(columns={0: "total Games"}, inplace=True)
ol_general.head(n=3)

,mean Summer,mean Winter,total Games
Afghanistan,1.000000,NaN,1.000000
Algeria,1.800000,NaN,1.800000
Argentina,1.857143,NaN,1.857143


### Añadiento la cantidad de veces que un ha participado en los juegos de verano e invierno y los totales en participaciones al dataframe del paso anterior 

In [66]:
ol_general=pd.merge(ol_general,pd.DataFrame(ol['№ Summer']),how='outer',left_index=True,right_index=True)
ol_general=pd.merge(ol_general,pd.DataFrame(ol['№ Winter']),how='outer',left_index=True,right_index=True)
ol_general=pd.merge(ol_general,pd.DataFrame(ol['№ Games']),how='outer',left_index=True,right_index=True)
ol_general.head(n=3)

,mean Summer,mean Winter,total Games,№ Summer,№ Winter,№ Games
Afghanistan,1.000000,NaN,1.000000,13,0,13
Algeria,1.800000,NaN,1.800000,12,3,15
Argentina,1.857143,NaN,1.857143,23,18,41


#### se desea dividir la media de juegos de verano e invierno sobre el total de veces que se ha participado. Como en los juegos de invierno algunos paises no participaron se debe poner la restriccion del cero al momento de dividir 

In [67]:
dc=ol_general[ol_general['№ Winter']!=0]    # elementos diferentes de cero para winter
mean_summer=dc['mean Summer']/dc['№ Summer']
mean_winter=dc['mean Winter']/dc['№ Winter']
mean_total=dc['total Games']/dc['№ Games']


# Dataframe final: este dataframe contiene nuestro dataset final, el cual tiene en cuenta la poblacion, el PIB y los juegos de verano e invierno. Todo con el fin de reponder a la pregunta, ¿Cuál ha sido el país más ganador en los olímpicos? 

Al momento de hacer el merge con el dataframe PIB y pobl es mejor hacer un how='outer' debido que existen algunos paises que no reportan PIB, pero si tienen competiciones en los juegos, y la idea es tambien analizar solos los juegos sin PIB ni poblacion, por tal razon se hace un how='outer'

In [75]:
dataframe_final=pd.merge(pd.DataFrame(mean_summer),pd.DataFrame(mean_winter),how='outer',left_index=True,right_index=True)
dataframe_final=pd.merge(dataframe_final,pd.DataFrame(mean_total),how='outer',left_index=True,right_index=True)
dataframe_final=pd.merge(dataframe_final,s_pobl,how='outer',left_index=True,right_index=True)
dataframe_final=pd.merge(dataframe_final,s_PIB,how='outer',left_index=True,right_index=True)


In [76]:
dataframe_final.rename(columns={'0_x': "mean Summer"}, inplace=True)
dataframe_final.rename(columns={'0_y': "mean Winter"}, inplace=True)
dataframe_final.rename(columns={0: "total Games"}, inplace=True)

# ¿Cuál ha sido el país más ganador en los olímpicos?

Existen muchos casos para dar respuesta a la pregunta, para nuestro caso se tendran en cuenta tres, que segun nuestra forma de ver son los mas interesantes

## 1. Teniendo en cuenta solo los juegos de verano 

Teniendo en cuenta solo los juegos de verano, donde se hace uso del método de clasificación de Fibonacci mencionado anteriormente

In [82]:
summer=dataframe_final.sort_values('mean Summer',ascending=False)   # Ordenando los paises con mayor cantidad de medallas
summer['mean Summer'].head(n=5)   # ESTOS SON LOS 5 PAISES CON MAYORES PUNTUACION TENIENDO EN CUENTA EL MÉTODO DE CLASIFICACIÓN

Unified Team              2.142857
Montenegro                1.000000
United Team of Germany    0.644068
Serbia and Montenegro     0.629630
Serbia                    0.523810
Name: mean Summer, dtype: float64

## 2. Teniendo en cuenta solo los juegos de invierno

In [84]:
winter=dataframe_final.sort_values('mean Winter',ascending=False)   # Ordenando los paises con mayor cantidad de medallas
winter['mean Winter'].head(n=5) # ESTOS SON LOS 5 PAISES CON MAYORES PUNTUACION TENIENDO EN CUENTA EL MÉTODO DE CLASIFICACIÓN

Unified Team              2.043478
United Team of Germany    0.719298
Uzbekistan                0.500000
Slovakia                  0.366667
Russia                    0.352151
Name: mean Winter, dtype: float64

## 3. Teniendo en cuenta el PIB y la cantidad de población 

In [87]:
# Medallas obtenidad divididas entre el PIB y la poblacion
PIB_pobl1=pd.DataFrame(dataframe_final['mean Summer']/dataframe_final['Poblacion media'] + dataframe_final['mean Summer']/dataframe_final['promedio PIB'])
PIB_pobl2=pd.DataFrame(dataframe_final['mean Winter']/dataframe_final['Poblacion media'] + dataframe_final['mean Winter']/dataframe_final['promedio PIB'])

In [104]:
PIB_pobl=pd.merge(PIB_pobl1,PIB_pobl2,how='outer',left_index=True,right_index=True)
PIB_pobl=PIB_pobl.loc[PIB_pobl.index].sum(axis=1)   
PIB_pobl.sort_values(ascending=False).head(n=5)  # ESTOS SON LOS 5 PAISES CON MAYORES PUNTUACION TENIENDO EN CUENTA EL
                                                 # EL PIB Y LA POBLACION

Liechtenstein    3.297501e-06
Tonga            2.712399e-06
Montenegro       1.724705e-06
Bermuda          1.019207e-06
Luxembourg       9.010476e-07
dtype: float64

# Como apunte, resulta interesante que segun los tre metodos planteados en ninguno en los 5 primeros puestos aparece EEUU y si se miran solo las cantidades de medalla de oro son los primeros 

In [112]:
ol=ol.sort_values('Gold',ascending=False)
ol['Gold'].head(n=5)

Totals           4809
United States     976
Soviet Union      395
Great Britain     236
France            202
Name: Gold, dtype: int64